In [ ]:
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!rm myscript.sh
!echo '#!/bin/bash' > myscript.sh
!echo 'ollama serve' >> myscript.sh

!chmod +x myscript.sh
!nohup ./myscript.sh &

rm: cannot remove 'myscript.sh': No such file or directory
nohup: appending output to 'nohup.out'


In [ ]:
!nohup ollama run mixtral:8x22b > ollama_model_instruct.log 2>&1 &

In [ ]:
!ollama list

In [ ]:
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="mixtral:8x22b", temperature=0.0)

In [ ]:
import pandas as pd
dev_data_df = pd.read_csv('dev_data_df.csv')

In [ ]:
from tqdm.auto import tqdm

questions = []

for i in tqdm(range(len(dev_data_df))):
    prompt_template = f"""
        Convert the following claim to one neutral question. Do not miss out anything important form the claim. Question the claim, not the fact.
        Look at the examples carefully and consturct the question accordingly:

        Example:

        Claim: 'Donald Trump has stated he will not contest for the next elections'
        Incorrect Question: 'What did Donald Trump state for the next elections?'
        Correct Question: 'Did Donald Trump state that he will not contest for the next elections?'

        Claim: 'The population of nigera is 2.5 million'
        Incorrect Question: 'What is the population of nigera?'
        Correct Question: 'Is the population of nigera 2.5 million?'

        Given Claim: {dev_data_df['claim'][i]}

        Write only the question you generate. Not anything else.
        """
    question = llm.invoke(prompt_template).strip()
    questions.append(question)

In [ ]:
import re

for q in questions:
    q = re.sub(r"^(Correct Question:|Incorrect Question:)\s*'|'\s*$|\"", "", q).strip()
    q = q.replace("Correct Question: ", "'")
    q = q.replace("\'", "'")
    q = q.replace("'", "")

In [ ]:
dev_data_df['questions'] = questions

In [ ]:
dev_data_df = dev_data_df.to_csv('dev_data_df.csv')

In [ ]:
import json
from pathlib import Path
from tqdm.auto import tqdm
from typing import List, Dict, Union, Optional

from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
from langchain_community.llms import Ollama

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!unzip top_three_dev.zip

In [ ]:
class JSONLLoader(BaseLoader):
    def __init__(
        self,
        file_path: Union[str, Path],
        content_key: Optional[str] = None,
        ):
        self.file_path = Path(file_path).resolve()
        self._content_key = content_key

    def load(self) -> List[Document]:
        docs = []
        with open(self.file_path, 'r', encoding="utf8") as file:
            for line in file:
                data = json.loads(line.strip())
                claim_id = data['claim_id']
                type_ = data['type']
                query = data['query']
                url = data['url']
                url2text = data['url2text']

                text = ' '.join(url2text)

                metadata = dict(
                    claim_id=claim_id,
                    type=type_,
                    query=query,
                    source=url
                )
                docs.append(Document(page_content=text, metadata=metadata))

        return docs

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
 ("system", "Based on the given statement, classify the given claim into one of the following four categories: 1) Supported - If there is sufficient evidence that the claim is legit, Refuted - Even if there is some evidence that the claim is fabricated or false, Not Enough Evidence - if there isn't sufficient evidence to either support or refute it, Conflicting Evidence/Cherrypicking - if the claim has both supporting and refuting evidence"),
 ("system", "Read the given statements carefully and judge whether they support the claim or refute it."),
 ("human", "Claim: {claim}"),
 ("human", "Statement: {relevant_sentence}"),
 ("human", "Pick only one from ['Supported', 'Refuted', 'Not Enough Evidence', 'Conflicting Evidence/Cherrypicking']")
])

In [ ]:
chain = prompt_template|llm|StrOutputParser()

In [ ]:
def get_relevant_sentence(question, evidence):
    chain = prompt_template|llm|StrOutputParser()
    return chain.invoke({"question":question, "evidence":evidence.page_content}).strip()

def generate_claim_dicts(claim: str, docs: List[Document], question: str, answers: List[str], pred_label: str) -> Dict:
    evidences = []

    for i in range(len(docs)):
        doc = docs[i]
        answer = answers[i]
        question = question
        source = doc.metadata['source']

        evidences.append({'question': question,'answer': answer,'url': source})

    claim_dict = {
        'claim_id': int(doc.metadata['claim_id']),
        'claim': claim,
        'evidence': evidences,
        'pred_label': pred_label
    }

    return claim_dict

In [ ]:
claim_dicts = []

for i in tqdm(range(len(dev_data_df))):
    claim = dev_data_df['claim'][i]
    question = dev_data_df['questions'][i]
    json_path = f'/content/top_three_dev/{i}.json'

    loader = JSONLLoader(file_path=json_path)
    data = loader.load()
    relevant_sentences = [get_relevant_sentence(question, evidence) for evidence in data]
    pred_label = chain.invoke({"claim":claim, "relevant_sentence":' '.join(relevant_sentences)}).strip()

    claim_dicts.append(generate_claim_dicts(claim, data, question, relevant_sentences, pred_label))

In [ ]:
with open('/content/drive/MyDrive/dev_preds.json', 'w') as file:
    json.dump(claim_dicts, file, indent=4)